In [1]:
import pandas as pd
import numpy as np
import datetime
from time import gmtime, strftime
import mariadb
from sqlalchemy import create_engine
import pymysql

In [2]:
def date_to_strdate(date):
    return     date.strftime('%Y-%m-%d %H:%M:%S')

In [3]:
pan_conn = pymysql.connect(host='ec1-it-prd-wibledrive-rds.clv6o5ibky6t.eu-central-1.rds.amazonaws.com', user='devops', password='Maas1223@' , charset='utf8')

In [4]:
core_conn = pymysql.connect(host='ec1-it-prd-core-rds.clv6o5ibky6t.eu-central-1.rds.amazonaws.com', user='devops', password='Maas1223@' , charset='utf8')

In [5]:
# 월요일 00시에 돌아간다는 가정. 아무튼 월요일에 돌아야함.
today = datetime.date.today() 
last_monday = today - datetime.timedelta(days=today.weekday()+7)
last_sunday = today - datetime.timedelta(days=today.weekday()+1)


In [6]:
start_date = last_monday.strftime("%Y-%m-%d %H:%M:%S")
print("data starting on:",start_date)

data starting on: 2022-01-10 00:00:00


In [7]:
end_date = last_sunday.strftime("%Y-%m-%d")
end_date = end_date[:11]+' 23:59:59'
print("data end on:",end_date)

data end on: 2022-01-16 23:59:59


# table 미리 불러오기 with pandas

In [8]:
print("get data from DB")
#전체 예약 정보
query = 'SELECT * FROM maas_db_reservation.reservations'
print(query)
reservations = pd.read_sql_query(query,core_conn)


get data from DB
SELECT * FROM maas_db_reservation.reservations


In [9]:
query = 'SELECT * FROM maasfms.reservation_short'
print(query)
reservation_short = pd.read_sql_query(query,pan_conn)

SELECT * FROM maasfms.reservation_short


In [10]:
#mp 테이블
query = "select p.reservation_id, (case when p.pg_tid is null then 'Offline' else 'Online' end) on_off from maasfms.payment p left join maasfms.payment_history ph on p.reservation_id = ph.reservation_id left join maasfms.reservation_short rs on p.reservation_id = rs.reservation_id where p.deleted_at is null and p.reservation_count between 0 and 1"
print(query)

mp = pd.read_sql_query(query,pan_conn)


select p.reservation_id, (case when p.pg_tid is null then 'Offline' else 'Online' end) on_off from maasfms.payment p left join maasfms.payment_history ph on p.reservation_id = ph.reservation_id left join maasfms.reservation_short rs on p.reservation_id = rs.reservation_id where p.deleted_at is null and p.reservation_count between 0 and 1


In [11]:
#전체 rental vehicle 테이블
query = 'SELECT * FROM maas_db_reservation.rental_vehicles where deleted_at is null'
print(query)

rental_vehicles = pd.read_sql_query(query,core_conn)


SELECT * FROM maas_db_reservation.rental_vehicles where deleted_at is null


In [12]:
query = "select * from maasfms.location where name not in ('Songdong-Gu', 'Seoul','KMIT','Test','이종원','KME')"
print(query)

location = pd.read_sql_query(query,pan_conn)
print("check location names in the system:",location.name.unique())
print()

select * from maasfms.location where name not in ('Songdong-Gu', 'Seoul','KMIT','Test','이종원','KME')
check location names in the system: ['KMG TOP s.r.o.' 'KIA Bratislava s.r.o' 'AUTO-VALAS s.r.o.'
 'Slovakia Auto, s.r.o.' 'Livorno' 'AUTOMOBILS A.R. - Sagrada Familia'
 'AUTOMOBILS A.R. - Zona Franca' 'AUTOMOBILS A.R. - Maquinista'
 'AUTOMOBILS A.R. - Sant Boi' 'ELECKAI MOTOR 3.0, S.L.' 'Tavernerio (CO)'
 'ELITE MOTORS - Kia Vénissieux' 'AMK - Kia Perpignan'
 'Milano V.Gallarate (MI)' 'San Salvo (CH)' 'AMK 31 - Kia Toulouse'
 'Grottammare' 'Santa Giusta' 'Rome' 'Solagna (VI)' 'EDENAUTO - Kia Pau'
 'Sassari' 'Pian Camuno' 'Riviera Car Center - Kia Le Cannet'
 'CAR Avenue - Kia Metz' 'Arcidosso' 'Milano V.Cassinis (MI)' 'Terni'
 'Olbia' 'San Giovanni Teatino (CH)' 'Daejeon' 'CAR Avenue - Kia Mulhouse'
 'Nuoro' 'Cantù (CO)' 'Lecce' 'Induno Olona (VA)'
 "KOTE AUTO - Kia Villeneuve d'Ascq" 'CARSERWIS' 'SZIC' 'KORCZYK PLUS'
 'Camerano (AN)' 'Saronno (VA)' 'Rieti' 'Foggia' 'Modugno (BA)'
 'Lura

In [13]:
# id컬럼명 수정
location["pickup_station_id"] = location["ID"]

In [14]:
#전체 vehicle
query = 'SELECT * FROM maas_db_vehicle.vehicles'
print(query)
vehicles = pd.read_sql_query(query,core_conn)
vehicles["business_id"] = vehicles["group_id"]


SELECT * FROM maas_db_vehicle.vehicles


In [15]:
#전체 vehicle model
query = 'SELECT * FROM `maas_db_vehicle-model`.vehicle_models '
print(query)
vehicle_models = pd.read_sql_query(query,core_conn)


SELECT * FROM `maas_db_vehicle-model`.vehicle_models 


In [16]:
vehicle_models1 = vehicle_models[vehicle_models["is_standard"]==0]


In [17]:
# 스탠다드 모델만 가져옴
print("get standard vehicle model")
vehicle_models2 = vehicle_models[vehicle_models["is_standard"]==1]
# vehicle_models2 = vehicle_models2[vehicle_models2["deleted_at"].isna()]


get standard vehicle model


In [18]:
query = "select * from maasfms.chargevehicle where deleted_at is null "
print(query)
chargevehicle = pd.read_sql_query(query, pan_conn)

select * from maasfms.chargevehicle where deleted_at is null 


In [19]:
query = "select * from maasfms.business where deleted_at is null"
print(query)
business = pd.read_sql_query(query, pan_conn)
business["business_id"] = business["id"]


select * from maasfms.business where deleted_at is null


In [20]:
query = "select cv.vehicle_id \
          from maasfms.chargevehicle cv \
          join maasfms.charge c on cv.charge_id = c.id \
         where 1 = 1 \
           and cv.deleted_at is null \
           and c.deleted_at is null"
print(query)
vehicle_charge = pd.read_sql_query(query, pan_conn)


select cv.vehicle_id           from maasfms.chargevehicle cv           join maasfms.charge c on cv.charge_id = c.id          where 1 = 1            and cv.deleted_at is null            and c.deleted_at is null


In [21]:
query = "select reservation_id , min(created_at) as reservated_at from maas_db_reservation.reservations where created_at >= '2020-09-20 22:00:00'            and created_at <= '"+ end_date+ "'       group by reservation_id"
print(query)
reservated_at = pd.read_sql_query(query, core_conn)


select reservation_id , min(created_at) as reservated_at from maas_db_reservation.reservations where created_at >= '2020-09-20 22:00:00'            and created_at <= '2022-01-16 23:59:59'       group by reservation_id


In [22]:
query = "select * from maasfms.country "
print(query)
country = pd.read_sql_query(query, pan_conn)

select * from maasfms.country 


In [23]:
vehicle_models1 = vehicle_models1.sort_values(by="created_at", ascending=True)
vehicle_models1 = vehicle_models1.drop_duplicates(subset=["vehicle_model_id","standard_model_id","vehicle_model_name"], keep="last")

In [24]:
vehicles1 = vehicles.drop_duplicates(subset=["vehicle_id","model_id"], keep="last")

In [25]:

print("============================ query end ==============================")
print("closing connections")
# pan_conn.close()
# core_conn.close()
print("============================ stat start ==============================")
print("예약 일수 별 예약 통계")
#reservations join rental vehicle
print("join reservations, rental_vehicles")
day_stat_reservation_count = pd.merge(reservations, rental_vehicles[['reservation_id','vehicle_id','pickup_station_id']], how='left', on='reservation_id')
print("row count of merged above: ",len(day_stat_reservation_count))
#reservation join rental vehilce inner join location 

print("join location")
day_stat_reservation_count = pd.merge(day_stat_reservation_count, location[["pickup_station_id","name"]], how='inner',on='pickup_station_id')
print("row count of merged above: ",len(day_stat_reservation_count))

print("join vehicles")
day_stat_reservation_count = pd.merge(day_stat_reservation_count, vehicles1[["vehicle_id","model_id"]], how="inner", on="vehicle_id")
print("row count of merged above: ",len(day_stat_reservation_count))
day_stat_reservation_count["vehicle_model_id"] = day_stat_reservation_count["model_id"]

print("join vehicle model")
day_stat_reservation_count = pd.merge(day_stat_reservation_count, vehicle_models1[["vehicle_model_id","standard_model_id","vehicle_model_name"]], how="inner", on="vehicle_model_id")
print("row count of merged above: ",len(day_stat_reservation_count))

print("join standard vehicle model")
day_stat_reservation_count = pd.merge(day_stat_reservation_count, vehicle_models2[["vehicle_model_name","standard_model_id"]], how="inner", on="standard_model_id")
print("row count of merged above: ",len(day_stat_reservation_count))

print("join mc(chargevheicle and charge)")
day_stat_reservation_count = pd.merge(day_stat_reservation_count, vehicle_charge, how="inner", on='vehicle_id')
print("row count of merged above: ",len(day_stat_reservation_count))
print("join reservated_at")
day_stat_reservation_count = pd.merge(day_stat_reservation_count, reservated_at, how="inner", on="reservation_id")
print("row count of merged above: ",len(day_stat_reservation_count))
# print("join mp")
# day_stat_reservation_count = pd.merge(day_stat_reservation_count, mp, how="left", on='reservation_id')
# print("row count of merged above: ",len(day_stat_reservation_count))
print()

============================ query end ==============================
closing connections
============================ stat start ==============================
예약 일수 별 예약 통계
join reservations, rental_vehicles
row count of merged above:  6190
join location
row count of merged above:  5917
join vehicles
row count of merged above:  5823
join vehicle model
row count of merged above:  5823
join standard vehicle model
row count of merged above:  5823
join mc(chargevheicle and charge)
row count of merged above:  5670
join reservated_at
row count of merged above:  5442



In [26]:
# 비교를 위한 컬럼 type 변경
day_stat_reservation_count["reserveated_at"] = pd.to_datetime(day_stat_reservation_count['reservated_at'])

In [27]:
#where 절

day_stat_reservation_count = day_stat_reservation_count[day_stat_reservation_count['reservated_at'] >= datetime.datetime.strptime('2020-09-20 22:00:00','%Y-%m-%d %H:%M:%S')]
day_stat_reservation_count = day_stat_reservation_count[day_stat_reservation_count['reservated_at'] <= datetime.datetime.strptime(end_date,'%Y-%m-%d %H:%M:%S')]
day_stat_reservation_count = day_stat_reservation_count[day_stat_reservation_count['deleted_at'].isna()]
day_stat_reservation_count = day_stat_reservation_count[day_stat_reservation_count['reservation_status'].isin(('CONFIRMED', 'INUSE', 'RETURNED'))]


In [28]:
#데이터 전처리 진행
day_stat_reservation_count = day_stat_reservation_count[day_stat_reservation_count['deleted_at'].isna()]
day_stat_reservation_count = day_stat_reservation_count.sort_values(by="reservated_at", ascending=False)
day_stat_reservation_count.reservation_start_time = day_stat_reservation_count.reservation_start_time.dt.date
day_stat_reservation_count.reservation_end_time  = day_stat_reservation_count.reservation_end_time.dt.date
day_stat_reservation_count["rental days"]= day_stat_reservation_count.reservation_end_time - day_stat_reservation_count.reservation_start_time
# day_stat_reservation_count = day_stat_reservation_count.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)

# 일자 별 예약일 합계 

In [29]:
print("1~7 days")
week = day_stat_reservation_count[(day_stat_reservation_count["rental days"]>=datetime.timedelta(days=0)) & (day_stat_reservation_count["rental days"]<= datetime.timedelta(days=6)) ].id.count()
print(week)
print("====================")
print("8~29 days")
month = day_stat_reservation_count[(day_stat_reservation_count["rental days"]>=datetime.timedelta(days=7)) & (day_stat_reservation_count["rental days"]<= datetime.timedelta(days=29)) ].id.count()
print(month)
print("====================")
print("1~3 months")
three_month = day_stat_reservation_count[(day_stat_reservation_count["rental days"]>=datetime.timedelta(days=30)) & (day_stat_reservation_count["rental days"]<= datetime.timedelta(days=89)) ].id.count()
print(three_month)
print("====================")
print("3~ months")
etc = day_stat_reservation_count[(day_stat_reservation_count["rental days"]>=datetime.timedelta(days=90))  ].id.count()
print(etc)
print("====================")
print("all")
print(len(day_stat_reservation_count))
day_reservation = {'~1week': [week], 

                  '~1M': [month], 

                 '1M~3M': [three_month],
                  
                  '3M~': [etc],
                  'SUM':[len(day_stat_reservation_count)]}
df = pd.DataFrame(day_reservation)



1~7 days
724
8~29 days
133
1~3 months
46
3~ months
6
all
909


# 차종별 예약 현황 - 국가별 신규 차량이 등록되고 있으므로 전과 데이터 상이 할 수 있음

In [30]:
print('--------------------------------------------------------')
print("차종별 예약 현황")
print("차종별 예약현황 data processing:")
print("left join reservations, rental_vhehicles")
vehicle_stat_reservation_count = pd.merge(reservations, rental_vehicles[["reservation_id","vehicle_id","pickup_station_id"]], how="left", on="reservation_id")
print("row count of merged above: ",len(vehicle_stat_reservation_count))

print("inner join location")
vehicle_stat_reservation_count = pd.merge(vehicle_stat_reservation_count, location[["pickup_station_id","name"]], how='inner',on='pickup_station_id')
print("row count of merged above: ",len(vehicle_stat_reservation_count))

print("inner join vehicles")
vehicle_stat_reservation_count = pd.merge(vehicle_stat_reservation_count, vehicles1[["vehicle_id","model_id"]], how="inner", on="vehicle_id")
vehicle_stat_reservation_count["vehicle_model_id"] = vehicle_stat_reservation_count["model_id"]
print("row count of merged above: ",len(vehicle_stat_reservation_count))
print("inner join vehicle models")

vehicle_stat_reservation_count2 = pd.merge(vehicle_stat_reservation_count, vehicle_models1[["vehicle_model_id","standard_model_id","vehicle_model_name"]], how="inner", on="vehicle_model_id")
print("row count of merged above: ",len(vehicle_stat_reservation_count2))

print("inner join standard vehicle models")
vehicle_stat_reservation_count2 = pd.merge(vehicle_stat_reservation_count2, vehicle_models2[["vehicle_model_name","standard_model_id"]], how="inner", on="standard_model_id")
print("row count of merged above: ",len(vehicle_stat_reservation_count2))

print("inner join mc(vehicle charge and charge)")
vehicle_stat_reservation_count2 = pd.merge(vehicle_stat_reservation_count2, vehicle_charge, how="inner", on="vehicle_id")
print("row count of merged above: ",len(vehicle_stat_reservation_count2))

print("inner join reservated_at")
vehicle_stat_reservation_count2 = pd.merge(vehicle_stat_reservation_count2, reservated_at, how="inner", on="reservation_id")
print("row count of merged above: ",len(vehicle_stat_reservation_count2))
print()

--------------------------------------------------------
차종별 예약 현황
차종별 예약현황 data processing:
left join reservations, rental_vhehicles
row count of merged above:  6190
inner join location
row count of merged above:  5917
inner join vehicles
row count of merged above:  5823
inner join vehicle models
row count of merged above:  5823
inner join standard vehicle models
row count of merged above:  5823
inner join mc(vehicle charge and charge)
row count of merged above:  5670
inner join reservated_at
row count of merged above:  5442



In [31]:
#where 절
vehicle_stat_reservation_count2 = vehicle_stat_reservation_count2[vehicle_stat_reservation_count2['reservated_at'] >= datetime.datetime.strptime('2020-09-20 22:00:00','%Y-%m-%d %H:%M:%S')]
vehicle_stat_reservation_count2 = vehicle_stat_reservation_count2[vehicle_stat_reservation_count2['reservated_at'] <= datetime.datetime.strptime(end_date,'%Y-%m-%d %H:%M:%S')]
vehicle_stat_reservation_count2 = vehicle_stat_reservation_count2[vehicle_stat_reservation_count2['deleted_at'].isna()]
vehicle_stat_reservation_count2 = vehicle_stat_reservation_count2[vehicle_stat_reservation_count2['reservation_status'].isin(('CONFIRMED', 'INUSE', 'RETURNED'))]


In [32]:
vehicle_stat_reservation_count2 = vehicle_stat_reservation_count2.sort_values(by="reservated_at", ascending=False)
# vehicle_stat_reservation_count2 = vehicle_stat_reservation_count2.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)


In [33]:
vehicle_stat_reservation_count2["this week count"]=np.where((vehicle_stat_reservation_count2["reservated_at"] >= datetime.datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')) &( vehicle_stat_reservation_count2["reservated_at"] <= datetime.datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')), 1, 0)
vehicle_stat_reservation_count2["tot count"] =1

# 차종별 예약 현황

,this week count,tot count
vehicle_model_name_y,,
Ceed CD,3,168
Ceed CD CUV,2,29
Ceed CD CUV MH,0,3
Ceed CD CUV PHEV,0,15
Ceed CD GT,1,25
Ceed SW CD,3,70
Niro DE PE,1,52
Niro DE PHEV,1,14
Niro DQ EV,1,81


In [34]:
vehicle_stat_reservation_count2_group = vehicle_stat_reservation_count2[["this week count","tot count"]].groupby(vehicle_stat_reservation_count2['vehicle_model_name_y'])
print("reservation count group by vehicle model")# 국가 별로 나눠야함 --> business country join해서 groupby에 country_name 추가
print(vehicle_stat_reservation_count2_group.sum())
print()

reservation count group by vehicle model
                      this week count  tot count
vehicle_model_name_y                            
Ceed CD                             3        168
Ceed CD CUV                         2         29
Ceed CD CUV MH                      0          3
Ceed CD CUV PHEV                    0         15
Ceed CD GT                          1         25
Ceed SW CD                          3         70
Niro DE PE                          1         52
Niro DE PHEV                        1         14
Niro DQ EV                          1         81
Picanto JA PE                       8        302
Sportage QL PE                      1         42
Sportage QL PE MH                   0          1
Stonic YB-CUV PE                    4        107



# 차량 등록 현황

### 이부분은 원래 가용차량 아닌 전체 차량이지만 그냥 스킵합시다 요즘제 등록안된

# 국가별 차량 등록 현황

In [35]:
print('--------------------------------------------------------')
print("국가별 차량 등록 현황")
print("국가별 차량 등록 현황 data processing:")
print("left join business, location")
business["business_id"] = business["id"]
country_vehicle = pd.merge(business, location, how="left",on="business_id")
print("row count of merged above: ",len(country_vehicle))

print("left join vehicles")
vehicles["business_id"] = vehicles["group_id"]
country_vehicle = pd.merge(country_vehicle, vehicles, how="left", left_on=["business_id","pickup_station_id"], right_on=["business_id", "station_id"])
print("row count of merged above: ",len(country_vehicle))

print("left join vehicle models")
country_vehicle["vehicle_model_id"] = country_vehicle["model_id"]
country_vehicle = pd.merge(country_vehicle, vehicle_models[["vehicle_model_id","standard_model_id","vehicle_model_name"]], how="inner", on="vehicle_model_id")
print("row count of merged above: ",len(country_vehicle))

print("left join standard vehicle model")
country_vehicle = pd.merge(country_vehicle, vehicle_models2[["vehicle_model_name","standard_model_id"]], how="inner", on="standard_model_id")
print("row count of merged above: ",len(country_vehicle))

print("left join country")
country_vehicle = pd.merge(country_vehicle, country[["id","country_name"]], how="inner", left_on=["country_id"], right_on=["id"] )
print("row count of merged above: ",len(country_vehicle))
country_vehicle = country_vehicle.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)


--------------------------------------------------------
국가별 차량 등록 현황
국가별 차량 등록 현황 data processing:
left join business, location
row count of merged above:  44
left join vehicles
row count of merged above:  168
left join vehicle models
row count of merged above:  403
left join standard vehicle model
row count of merged above:  3715
left join country
row count of merged above:  3715


In [36]:
country_vehicle = country_vehicle[
    (country_vehicle.deleted_at.isna()) &
    (country_vehicle.deleted_at_x.isna()) &
    (country_vehicle.deleted_at_y.isna())
]


In [37]:
print("")
print("국가, 모델(standard) 별 차량 등록 현황")
print(country_vehicle[['country_name', 'vehicle_model_name_y','id_x']].groupby(['country_name', 'vehicle_model_name_y']).count())


국가, 모델(standard) 별 차량 등록 현황
                                   id_x
country_name vehicle_model_name_y      
France       e-Niro                   2
Italia       Ceed CD                 10
             Ceed CD CUV              5
             Ceed CD CUV MH           2
             Ceed CD CUV PHEV         2
             Ceed CD GT               1
             Ceed SW CD               5
             Niro DE PE               6
             Niro DE PHEV             1
             Niro DQ EV               2
             Picanto JA PE           14
             Soul EV SK3              1
             Sportage QL PE           5
             Sportage QL PE MH        2
             Stonic YB-CUV PE         8
Slovensko    EV6                      1
             Sportage MY21            4


In [38]:
print("국가, 딜러그룹, 딜러쉽 별 차량 등록 현황")
print(country_vehicle[['country_name', 'name_x','created_at_x','name_y','created_at_y','id_x']].groupby(['country_name', 'name_x','created_at_x','name_y','created_at_y']).count())

국가, 딜러그룹, 딜러쉽 별 차량 등록 현황
                                                                                                            id_x
country_name name_x                      created_at_x        name_y                    created_at_y             
France       CHABRIER - Toulouse         2021-12-10 16:35:55 AMK 31 - Kia Toulouse     2021-12-15 14:53:57     2
Italia       CAR LISI                    2021-03-30 07:41:57 Foggia                    2021-04-19 11:02:48     4
             CLERICI AUTO SPA            2020-09-07 07:23:41 Cantù (CO)                2020-09-07 07:47:11     2
                                                             Induno Olona (VA)         2020-09-07 07:46:41     6
                                                             Lurate Caccivio (CO)      2020-09-07 07:44:44     5
                                                             Milano V.Cassinis (MI)    2020-09-07 07:40:15     2
                                                             Milano V.G

# MP online offline

In [39]:
print('--------------------------------------------------------')
print("결제 방법 통계(Online, Offline")
print("on-off line data processing:")
print("left join reservations, business")
mp_reservations = reservations[reservations["deleted_at"].isna()]
mp_reservations = pd.merge(mp_reservations, business[['name','business_id']], how='inner',left_on="group_id", right_on='business_id')
print("row count of merged above: ",len(mp_reservations))

print("left join maasfms payment")
mp_reservations = pd.merge(mp_reservations, mp, how='left',on="reservation_id")
print("row count of merged above: ",len(mp_reservations))

print("left join reservated_at")
mp_reservations = pd.merge(mp_reservations, reservated_at, how='left',on='reservation_id' )
print("row count of merged above: ",len(mp_reservations))

--------------------------------------------------------
결제 방법 통계(Online, Offline
on-off line data processing:
left join reservations, business
row count of merged above:  1384
left join maasfms payment
row count of merged above:  4660
left join reservated_at
row count of merged above:  4660


In [40]:
#where 절
mp_reservations = mp_reservations[mp_reservations['created_at'] >= datetime.datetime.strptime('2020-09-20 22:00:00','%Y-%m-%d %H:%M:%S')]
mp_reservations = mp_reservations[mp_reservations['reservation_status'].isin(('CONFIRMED', 'INUSE', 'RETURNED'))]
mp_reservations = mp_reservations[mp_reservations['reservated_at'] >= datetime.datetime.strptime(start_date,'%Y-%m-%d %H:%M:%S')]
mp_reservations = mp_reservations[mp_reservations['reservated_at'] <= datetime.datetime.strptime(end_date,'%Y-%m-%d %H:%M:%S')]


In [41]:
mp_reservations = mp_reservations.drop_duplicates(subset=["reservation_id"], keep="last")
mp_reservations.on_off = mp_reservations.on_off.fillna('Offline')
print()
print("결제 방법 별 수")
print(mp_reservations[["on_off","reservation_id"]].groupby('on_off').count())


결제 방법 별 수
         reservation_id
on_off                 
Offline               4
Online               21


In [42]:
print(mp_reservations[["on_off","reservation_id"]])


       on_off                        reservation_id
3466   Online  84d1eff1-71f8-11ec-9b92-cad5aee6311e
3470   Online  248c9840-71ea-11ec-bb3a-0e85c43bdb6c
3473   Online  9a34ec4b-72c0-11ec-bb3a-0e85c43bdb6c
3477   Online  9f0171c9-72b7-11ec-9b92-cad5aee6311e
3481   Online  8d17a04a-72af-11ec-bb3a-0e85c43bdb6c
3485   Online  0e85b60b-7234-11ec-9b92-cad5aee6311e
3487   Online  b5cbaed0-73ae-11ec-9b92-cad5aee6311e
3495   Online  392e3fc6-73ad-11ec-9b92-cad5aee6311e
3499   Online  9a09079e-72ca-11ec-bb3a-0e85c43bdb6c
3503   Online  9b228f1d-72bf-11ec-bb3a-0e85c43bdb6c
3507   Online  ef0993e7-7447-11ec-9b92-cad5aee6311e
3508  Offline  d592fc62-749f-11ec-bb3a-0e85c43bdb6c
3510  Offline  0bf452d1-72bb-11ec-bb3a-0e85c43bdb6c
3511   Online  60180975-7526-11ec-bb3a-0e85c43bdb6c
3515   Online  1b63bd52-72bb-11ec-bb3a-0e85c43bdb6c
3519   Online  e210535d-7515-11ec-bb3a-0e85c43bdb6c
3520  Offline  dd0b3831-7551-11ec-9b92-cad5aee6311e
3521  Offline  21f1f02c-7513-11ec-bb3a-0e85c43bdb6c
3810   Onlin

# 예약 종류별 통계

In [43]:
print('--------------------------------------------------------')
print("예약 종류별 통계")
print("reservation type data processing:")
print("join reservations, rental_vehicles")
regist_type_stat = pd.merge(reservations, rental_vehicles[['reservation_id','vehicle_id','pickup_station_id']], how='left', on='reservation_id')
print("row count of merged above: ",len(regist_type_stat))
#reservation join rental vehilce inner join location 

print("join location")
regist_type_stat = pd.merge(regist_type_stat, location[["pickup_station_id","name"]], how='inner',on='pickup_station_id')
print("row count of merged above: ",len(regist_type_stat))

print("join vehicles")
regist_type_stat = pd.merge(regist_type_stat, vehicles1[["vehicle_id","model_id"]], how="inner", on="vehicle_id")
print("row count of merged above: ",len(regist_type_stat))
regist_type_stat["vehicle_model_id"] = regist_type_stat["model_id"]

print("join vehicle model")
regist_type_stat = pd.merge(regist_type_stat, vehicle_models1[["vehicle_model_id","standard_model_id","vehicle_model_name"]], how="inner", on="vehicle_model_id")
print("row count of merged above: ",len(regist_type_stat))

print("join standard vehicle model")
regist_type_stat = pd.merge(regist_type_stat, vehicle_models2[["vehicle_model_name","standard_model_id"]], how="inner", on="standard_model_id")
print("row count of merged above: ",len(regist_type_stat))

print("join mc(chargevheicle and charge)")
regist_type_stat = pd.merge(regist_type_stat, vehicle_charge, how="inner", on='vehicle_id')
print("row count of merged above: ",len(regist_type_stat))

print("join reservation_short")
reservation_short = reservation_short[reservation_short['deleted_at'].isna()]
regist_type_stat = pd.merge(regist_type_stat, reservation_short, how="left", on="reservation_id")
print("row count of merged above:", len(regist_type_stat))

print("join reservated_at")
regist_type_stat = pd.merge(regist_type_stat, reservated_at, how="inner", on="reservation_id")
print("row count of merged above: ",len(regist_type_stat))



--------------------------------------------------------
예약 종류별 통계
reservation type data processing:
join reservations, rental_vehicles
row count of merged above:  6190
join location
row count of merged above:  5917
join vehicles
row count of merged above:  5823
join vehicle model
row count of merged above:  5823
join standard vehicle model
row count of merged above:  5823
join mc(chargevheicle and charge)
row count of merged above:  5670
join reservation_short
row count of merged above: 5673
join reservated_at
row count of merged above:  5445


In [44]:
regist_type_stat

,id,created_at_x,updated_at_x,deleted_at_x,creator_id,service_id,provider_id,reservation_id,product_id,reservation_start_time_x,...,installment_payment_pay,installment_payment_iterations,installment_payment_productname,installment_payment_rental_fee,enroll_status,charge_changed_yn,old_reservation_id,old_reservation_key,stripe_success_yn,reservated_at
0,62125,2021-05-13 08:05:37,2021-05-13 08:05:37,2021-05-13 08:05:39,,4d82f76c-3073-11ec-861d-0270ee4ad24a,7f87000d-142f-11ec-86a7-02ff1f7ba654,004ccff9-b3c2-11eb-857a-0a73dbc7f556,16bdb534-9913-11eb-a891-0a73dbc7f556,2021-05-13 08:10:00,...,0.0,0,None,30.0,None,NaN,None,None,1,2021-05-13 08:05:37
1,62126,2021-05-13 08:05:39,2021-05-13 08:05:39,2021-05-13 08:05:49,,4d82f76c-3073-11ec-861d-0270ee4ad24a,7f87000d-142f-11ec-86a7-02ff1f7ba654,004ccff9-b3c2-11eb-857a-0a73dbc7f556,16bdb534-9913-11eb-a891-0a73dbc7f556,2021-05-13 08:10:00,...,0.0,0,None,30.0,None,NaN,None,None,1,2021-05-13 08:05:37
2,62127,2021-05-13 08:05:49,2021-05-13 08:05:49,2021-05-13 15:03:51,,4d82f76c-3073-11ec-861d-0270ee4ad24a,7f87000d-142f-11ec-86a7-02ff1f7ba654,004ccff9-b3c2-11eb-857a-0a73dbc7f556,16bdb534-9913-11eb-a891-0a73dbc7f556,2021-05-13 08:05:49,...,0.0,0,None,30.0,None,NaN,None,None,1,2021-05-13 08:05:37
3,62128,2021-05-13 15:03:51,2021-05-13 15:03:51,NaT,,4d82f76c-3073-11ec-861d-0270ee4ad24a,7f87000d-142f-11ec-86a7-02ff1f7ba654,004ccff9-b3c2-11eb-857a-0a73dbc7f556,16bdb534-9913-11eb-a891-0a73dbc7f556,2021-05-13 08:05:49,...,0.0,0,None,30.0,None,NaN,None,None,1,2021-05-13 08:05:37
4,63375,2021-05-24 13:59:19,2021-05-24 13:59:19,2021-05-24 13:59:21,,4d82f76c-3073-11ec-861d-0270ee4ad24a,7f87000d-142f-11ec-86a7-02ff1f7ba654,3c0e1515-bc98-11eb-857a-0a73dbc7f556,16bdb534-9913-11eb-a891-0a73dbc7f556,2021-05-24 16:00:00,...,0.0,0,None,60.0,None,NaN,None,None,1,2021-05-24 13:59:19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5440,65873,2021-09-21 09:09:55,2021-09-21 09:09:55,2021-09-21 09:10:54,,4d82f76c-3073-11ec-861d-0270ee4ad24a,7f870a28-142f-11ec-86a7-02ff1f7ba654,aeb0d50d-1abb-11ec-916b-0a73dbc7f556,4001bf56-0981-11ec-993f-02f92df357c4,2021-09-21 09:10:00,...,0.0,0,,2061.9,,0.0,None,None,0,2021-09-21 09:09:53
5441,65874,2021-09-21 09:10:54,2021-09-21 09:10:54,2021-09-21 09:10:55,,4d82f76c-3073-11ec-861d-0270ee4ad24a,7f870a28-142f-11ec-86a7-02ff1f7ba654,aeb0d50d-1abb-11ec-916b-0a73dbc7f556,4001bf56-0981-11ec-993f-02f92df357c4,2021-09-21 09:10:00,...,0.0,0,,2061.9,,0.0,None,None,0,2021-09-21 09:09:53
5442,65875,2021-09-21 09:10:55,2021-09-21 09:10:55,2021-10-12 14:54:53,,4d82f76c-3073-11ec-861d-0270ee4ad24a,7f870a28-142f-11ec-86a7-02ff1f7ba654,aeb0d50d-1abb-11ec-916b-0a73dbc7f556,4001bf56-0981-11ec-993f-02f92df357c4,2021-09-21 09:10:55,...,0.0,0,,2061.9,,0.0,None,None,0,2021-09-21 09:09:53
5443,65876,2021-10-12 14:54:53,2021-10-12 14:54:53,2021-11-30 15:53:45,,4d82f76c-3073-11ec-861d-0270ee4ad24a,7f870a28-142f-11ec-86a7-02ff1f7ba654,aeb0d50d-1abb-11ec-916b-0a73dbc7f556,4001bf56-0981-11ec-993f-02f92df357c4,2021-09-21 09:10:55,...,0.0,0,,2061.9,,0.0,None,None,0,2021-09-21 09:09:53


In [45]:
#where 절
regist_type_stat = regist_type_stat[regist_type_stat['deleted_at_x'].isna()]
regist_type_stat = regist_type_stat[regist_type_stat['reservation_status_x'].isin(('CONFIRMED', 'INUSE', 'RETURNED'))]
regist_type_stat = regist_type_stat[regist_type_stat['reservated_at'] >= datetime.datetime.strptime('2020-09-20 22:00:00','%Y-%m-%d %H:%M:%S')]
regist_type_stat = regist_type_stat[regist_type_stat['reservated_at'] <= datetime.datetime.strptime(end_date,'%Y-%m-%d %H:%M:%S')]



In [46]:
regist_type_stat[["reservation_id","reservation_regist_type","reservated_at"]].groupby('reservation_regist_type').count()

,reservation_id,reservated_at
reservation_regist_type,,
B2B,16,16
COURTESY_CAR,101,101
OTHERS,214,214
PRIVATE,385,385
TEST_DRIVE,2,2
UNKNOWN_RESERVATION_REGIST_TYPE,192,192


# 가동률은

In [47]:
print('--------------------------------------------------------')
print("가동률")
activations = reservations[reservations['reservation_end_time'] >= reservations['reservation_start_time']]
activations = activations[activations['reservation_status'].isin(('CONFIRMED', 'INUSE', 'RETURNED'))]
activations = activations[activations["deleted_at"].isna()]                           
activations = activations[activations['reservation_start_time'] <= datetime.datetime.strptime(end_date,'%Y-%m-%d %H:%M:%S')]
activations = activations[activations['reservation_end_time'] >= datetime.datetime.strptime(start_date,'%Y-%m-%d %H:%M:%S')]


--------------------------------------------------------
가동률


In [48]:
activations['activations_start'] = ''
activations['activations_end'] = ''
for i in range(len(activations['reservation_start_time'])):
    if activations['reservation_start_time'].iloc[i]<= datetime.datetime.strptime(start_date,'%Y-%m-%d %H:%M:%S'):
        activations['activations_start'].iloc[i] = datetime.datetime.strptime(start_date,'%Y-%m-%d %H:%M:%S')
    else: 
        activations['activations_start'].iloc[i] = activations['reservation_start_time'].iloc[i]
        
    if activations['reservation_end_time'].iloc[i] > datetime.datetime.strptime(end_date,'%Y-%m-%d %H:%M:%S'):
        activations['activations_end'].iloc[i] = datetime.datetime.strptime(end_date,'%Y-%m-%d %H:%M:%S')
    else: 
        activations['activations_end'].iloc[i] = activations['reservation_end_time'].iloc[i]
        
        
activations.activations_start = pd.to_datetime(activations.activations_start)
activations.activations_end  = pd.to_datetime(activations.activations_end)
activations["rental days"]= (activations.activations_end - activations.activations_start).dt.days + 1
activation_calculate = pd.DataFrame({
    "총 예약일 수" : [activations["rental days"].sum()],
})
activation_calculate

C:\Users\E681013\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,총 예약일 수
0,189


# 파일로

In [53]:
print("========================== data processing end extracting to excel file ==========================")
print("extracting data to excel")
today = strftime("%Y-%m-%d", gmtime())
name = "주1회 보고용 통계 " + today

writer = pd.ExcelWriter(name+".xlsx", engine = 'xlsxwriter')
df.to_excel(writer, sheet_name="day_stat_reservation_count")
vehicle_stat_reservation_count2_group.sum().to_excel(writer, sheet_name="차종별 등록 현황")
country_vehicle[['country_name', 'vehicle_model_name_y','id_x']].groupby(['country_name', 'vehicle_model_name_y']).count().to_excel(writer, sheet_name="차량 등록 현황")
country_vehicle[['country_name', 'name_x','created_at_x','name_y','created_at_y','id_x']].groupby(['country_name', 'name_x','created_at_x','name_y','created_at_y']).count().to_excel(writer, sheet_name="차량 등록 현황2")
mp_reservations[["on_off","reservation_id"]].groupby('on_off').count().to_excel(writer, sheet_name="mp_on_off")
regist_type_stat[["reservation_id","reservation_regist_type","reservated_at"]].groupby('reservation_regist_type').count().to_excel(writer, sheet_name="regist_type")
activation_calculate.to_excel(writer, sheet_name="가동률 예약일 수")

writer.save()
writer.close()

========================== data processing end extracting to excel file ==========================
extracting data to excel


C:\Users\E681013\Anaconda3\lib\site-packages\xlsxwriter\workbook.py:329: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
